In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.filters import gaussian, threshold_otsu, threshold_multiotsu, sobel
from skimage.morphology import remove_small_objects, disk, binary_closing
from scipy.ndimage import zoom, binary_dilation, binary_erosion, distance_transform_edt
from skimage.measure import label, regionprops
from skimage import io, exposure, color
from skimage import measure, morphology
from skimage import exposure
from czifile import imread
from cellpose import models, plot 
from matplotlib.ticker import MaxNLocator
model = models.Cellpose(gpu=False, model_type='cyto3')

In [7]:
MIN_INCLUSION_SIZE = 10
MAX_INCLUSION_SIZE = 2000
RADIUS = 25

In [8]:
def display_image(image, path, type):
    """Display the image."""
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"{path} {type}")
    plt.show()

def get_filename_from_path(path):
    """Extracts the last part of the path after the final slash."""
    return os.path.basename(path)

def display_two_images(image1, image2, title1, title2, path):
    """Display two images side-by-side with smaller title font."""
    filename = os.path.basename(path)  # Extract final part of path

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    axes[0].imshow(image1, cmap='gray' if image1.ndim == 2 else None)
    axes[0].set_title(f"{filename} {title1}", fontsize=10)
    axes[0].axis('off')

    axes[1].imshow(image2, cmap='gray' if image2.ndim == 2 else None)
    axes[1].set_title(f"{filename} {title2}", fontsize=10)
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

def extract_image_paths(folder):
    """Extract all image file paths from the specified folder."""
    return [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

def read_image(image_path):
    """Read the LSM image from the specified path."""
    return imread(image_path)

def count(mask): 
    """Count the number of unique labels in the mask."""
    return len(np.unique(label(mask))) - 1  # Exclude background label (0)

def extract_channels(image: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Extract green and red channels from the squeezed image (shape: [Z, C, H, W]).""" 
    return image[0], image[1], image[2]  

def preprocess_green_channel(green_channel):
    """
    Preprocess the green fluorescence channel for better segmentation and inclusion detection.
    - Applies Gaussian blur to reduce noise.
    - Enhances contrast using sigmoid adjustment.
    - Normalizes intensities to [0, 1] for consistent processing.
    """
    confocal_img = gaussian(green_channel, sigma=2)
    confocal_img = exposure.adjust_sigmoid(green_channel, cutoff=0.25)
    confocal_img = normalize_image(confocal_img)
    return confocal_img

    return circular_mask, non_circular_mask
def normalize_image(image):
    """
    Normalize the image to the range [0, 1].
    This is useful for consistent processing across different images.
    """
    return (image - np.min(image)) / (np.max(image) - np.min(image))

def calculate_surface_area(labeled_image: np.ndarray) -> float:
    """Calculate the total surface area for labeled regions."""
    props = regionprops(labeled_image)
    return sum(prop.area for prop in props)


def split_by_circularity(binary_mask: np.ndarray, threshold: float):
    """
    Splits objects in a binary mask based on circularity.
    
    Parameters:
        binary_mask (ndarray): Binary image with objects (1) and background (0).
        threshold (float): Circularity threshold (0 to 1). Higher is more circular.
        
    Returns:
        circular_mask (ndarray): Mask of objects above threshold.
        non_circular_mask (ndarray): Mask of objects below or equal to threshold.
    """
    labeled = label(binary_mask)
    circular_mask = np.zeros_like(binary_mask, dtype=bool)
    non_circular_mask = np.zeros_like(binary_mask, dtype=bool)

    for region in regionprops(labeled):

        perimeter = region.perimeter or 1  # Avoid divide-by-zero
        circularity = 4 * np.pi * region.area / (perimeter ** 2)

        if circularity > threshold:
            circular_mask[labeled == region.label] = True
        else:
            non_circular_mask[labeled == region.label] = True

    return circular_mask, non_circular_mask

def segment_cells(green_channel):
    """
    Segment whole cells in the green channel using Cellpose.
    - Normalizes image intensity.
    - Suppresses bright spots (e.g., inclusions) to better detect cell boundaries.
    - Applies Gaussian blur for smoother segmentation input.
    - Gradually increases segmentation diameter until at least one cell is detected.
    """
    green_channel = normalize_image(green_channel)
    percentile_99 = np.percentile(green_channel, 99)
    
    # Suppress very bright pixels (inclusions)
    green_channel_remove_inclusions = np.where(green_channel < percentile_99, green_channel, percentile_99)
    green_channel_remove_inclusions = gaussian(green_channel_remove_inclusions, sigma=5)

    # Normalize again after processing
    green_channel_remove_inclusions = normalize_image(green_channel_remove_inclusions)

    # Try different diameters until cells are detected
    diameter = 150
    while diameter < 500:
        masks, flows, styles, diams = model.eval(green_channel_remove_inclusions, diameter=diameter, channels=[0, 0])
        labeled_cells = label(masks)
        if np.max(labeled_cells) > 0:
            return labeled_cells
        diameter += 25

    # No cells found
    return None

def extract_inclusions(green_channel, mask, display_graph=False):
    """
    Extract potential inclusions inside a cell.
    - Blurs and masks the cell region.
    - Computes intensity statistics for thresholding.
    - Applies different threshold strategies depending on intensity distribution.
    - Removes objects that are too small or too large to be inclusions.
    - Optionally shows histogram for debugging.
    """
    applied_mask_blurred = gaussian(green_channel, sigma=1) * mask
    applied_mask_eliminate_background = applied_mask_blurred[applied_mask_blurred > 0]

    # Normalize the signal within the masked region
    applied_mask_eliminate_background = normalize_image(applied_mask_eliminate_background)


    # Compute descriptive statistics for intensity distribution
    q3 = np.percentile(applied_mask_eliminate_background, 75)
    hist, bin_edges = np.histogram(applied_mask_eliminate_background, bins='fd')
    applied_mask = normalize_image(green_channel) * mask


    # Decide on thresholding strategy based on upper quartile
    if q3 < 0.4 and len(bin_edges) > 20:
        #threshold = max(threshold_otsu(applied_mask), 0.5)
        threshold = max(threshold_otsu(applied_mask), 0.5)
    else:
        threshold = 0.95

    # Apply threshold and size-based filters
    #threshold = max(threshold_otsu(applied_mask), 0.38)
    inclusions = applied_mask > threshold
    inclusions = remove_small_objects(inclusions, min_size=MIN_INCLUSION_SIZE)
    #inclusions = inclusions ^ remove_small_objects(inclusions, min_size=MAX_INCLUSION_SIZE)


    # Optional histogram display
    if display_graph:
        print("Threshold: ", threshold)
        print("Bin count", len(bin_edges))
        plt.hist(applied_mask_eliminate_background, bins='fd')
        plt.axvline(q3, color='purple', linestyle='dashed', linewidth=2, label=f'Q3: {q3:.2f}')
        plt.legend()
        plt.title("Intensity histogram")
        plt.show()


    return inclusions

def generate_inclusion_image(green_channel, labeled_cells):
    """
    Generate a binary image with all inclusions from all cells.
    - Loops through each segmented cell.
    - Extracts inclusions from each cell region.
    - Combines all into one final binary image.
    """
    inclusion_image = np.zeros_like(green_channel)

    for i, cell in enumerate(regionprops(labeled_cells)):
        if cell.area < 100:
            continue
        mask = labeled_cells == cell.label
        inclusions = extract_inclusions(green_channel, mask)
        inclusion_image += inclusions  # adds binary inclusion mask

    return inclusion_image


def filter_objects_by_overlap(source_mask, target_mask):
    """
    Keeps objects in `source_mask` that overlap with `target_mask`.

    Parameters:
        source_mask (ndarray): Binary mask with objects to test (e.g. clustered mitochondria).
        target_mask (ndarray): Binary mask defining inclusion region.

    Returns:
        filtered_mask (ndarray): Binary mask of source objects overlapping with target.
    """
    labeled_source = label(source_mask)
    filtered_mask = np.zeros_like(source_mask, dtype=bool)

    for region in regionprops(labeled_source):
        coords = tuple(region.coords.T)
        if np.any(target_mask[coords]):
            filtered_mask[coords] = True

    return filtered_mask


def get_overlapping_objects_mask(mask1, mask2):
    """
    Given two labeled masks (mask1 and mask2), return a binary mask containing
    all objects from both masks that overlap with each other.
    
    Parameters:
        mask1 (ndarray): Labeled mask (e.g., from skimage.label).
        mask2 (ndarray): Labeled mask.
    
    Returns:
        overlapping_mask (ndarray): Binary mask of the overlapping objects from both inputs.
    """
    overlap = (mask1 > 0) & (mask2 > 0)
    if not np.any(overlap):
        return np.zeros_like(mask1, dtype=np.uint8)  # No overlap

    overlapping_labels_1 = np.unique(mask1[overlap])
    overlapping_labels_2 = np.unique(mask2[overlap])

    # Create masks for overlapping objects
    mask1_overlap = np.isin(mask1, overlapping_labels_1)
    mask2_overlap = np.isin(mask2, overlapping_labels_2)

    combined_overlap_mask = mask1_overlap | mask2_overlap
    return combined_overlap_mask.astype(np.uint8)

def calculate_densities(inclusion_image, cell, channel):
    mask = cell > 0
    # dilate inclusion_image
    channel = channel * mask
    dilated_inclusion_image = binary_dilation(inclusion_image, disk(RADIUS))
    dilated_inclusion_image = dilated_inclusion_image * mask
    dilated_inclusion_image_donut = dilated_inclusion_image.astype(bool) ^ inclusion_image.astype(bool)
    overlap_with_channel = channel * dilated_inclusion_image_donut
    surface_area_overlap_within_donut = calculate_surface_area(label(overlap_with_channel))

    #display_image(inclusion_image, "inclusion_image", "mask")
    #display_image(binary_dilation(inclusion_image, disk(RADIUS)), "dilated_inclusion_image", "mask")
    #display_image(mask, "mask", "mask")
    #display_image(dilated_inclusion_image, "dilated_inclusion_image", "mask")
    #display_image(dilated_inclusion_image_donut, "dilated_inclusion_image_donut", "mask")


    cell_without_dilated_inclusions = mask ^ dilated_inclusion_image
    overlap_with_rest_of_the_cells = channel * cell_without_dilated_inclusions
    surface_area_overlap_without_donut = calculate_surface_area(label(overlap_with_rest_of_the_cells))
    #display_image(cell_without_dilated_inclusions, "cell_without_dilated_inclusions", "mask")
    #display_image(overlap_with_channel, "overlap_with_channel", "mask")
    #display_image(overlap_with_rest_of_the_cells, "overlap_with_rest_of_the_cells", "mask")
    

    return surface_area_overlap_within_donut, surface_area_overlap_without_donut


In [9]:
def dox_analysis(red:np.ndarray, orange:np.ndarray, green:np.ndarray, path:str) -> pd.DataFrame:
    data = []
    df_cell_summary = pd.DataFrame()

    print(f"Starting analysis on {path}")

    # Preprocess the green channel
    green = preprocess_green_channel(green)

    

    #return green

    labeled_cells = segment_cells(green)
    #display_image(labeled_cells, path, "Labeled Cells")

    inclusion_image = generate_inclusion_image(green, labeled_cells)

    #display_two_images(green, inclusion_image, "Green Channel", "Inclusion Image", path)

    contrast_adjusted_red_normalized = (red - red.min()) / (red.max() - red.min())
    threshold_value_red= np.mean(contrast_adjusted_red_normalized) + (np.std(contrast_adjusted_red_normalized) * 3) #1.75 works well
    red_thresholded = contrast_adjusted_red_normalized > threshold_value_red
    red_thresholded = remove_small_objects(red_thresholded, min_size=10)
    #display_two_images(red, red_thresholded, "Red Channel", "Red Thresholded", path)


    #contrast_adjusted_blue_normalized = (blue - blue.min()) / (blue.max() - blue.min())
    #threshold_value_blue= np.mean(contrast_adjusted_blue_normalized) + (np.std(contrast_adjusted_blue_normalized) * 1.5) #1.75 works well
    #blue_thresholded = contrast_adjusted_blue_normalized > threshold_value_blue
    #blue_thresholded = remove_small_objects(blue_thresholded, min_size=10)
    if ('mitotracker_farred_FCCP' in path): 
        contrast_adjusted_orange_normalized = (orange - orange.min()) / (orange.max() - orange.min())
        threshold_value_orange= np.mean(contrast_adjusted_orange_normalized) + (np.std(contrast_adjusted_orange_normalized) * 4) #1.75 works well
        orange_thresholded = contrast_adjusted_orange_normalized > threshold_value_orange
        orange_thresholded = remove_small_objects(orange_thresholded, min_size=10)
    else:
        threshold_value_orange  = threshold_otsu(orange)
        orange_thresholded = orange > threshold_value_orange
    #display_two_images(blue, blue_thresholded, "Blue Channel", "Blue Thresholded", path)

    cell_count = 1
    for i, cell in enumerate(regionprops(labeled_cells)):
        if cell.area < 100:  # Skip tiny regions likely to be noise
            continue

                # Create a mask for the current cell
        mask = labeled_cells == cell.label

        #inclusion within cell
        inclusion_mask = inclusion_image * mask
        red_mask = red_thresholded * mask
        orange_mask = orange_thresholded * mask

        red_surface_area = calculate_surface_area(label(red_mask))
        orange_surface_area = calculate_surface_area(label(orange_mask))

        red_surface_area_within_donut, red_surface_area_without_donut = calculate_densities(inclusion_mask, mask, red_thresholded)
        orange_surface_area_within_donut, orange_surface_area_without_donut = calculate_densities(inclusion_mask, mask, orange_thresholded)

        orange_red_overlap = get_overlapping_objects_mask(label(red_mask), label(orange_mask))

        orange_red_overlap_surface_area = calculate_surface_area(orange_red_overlap)
        orange_red_overlap_surface_area_within_donut, orange_red_overlap_surface_area_without_donut = calculate_densities(inclusion_mask, mask, orange_red_overlap)
        


        data.append({
            'Image Path': path,
            'Cell Number': cell_count,
            'Red surface area in all cell': red_surface_area,
            'Red surface area around inclusions': red_surface_area_within_donut,
            'Red surface area in remaining cytoplasm': red_surface_area_without_donut,
            'Orange surface area in all cell': orange_surface_area,
            'Orange surface area around inclusions': orange_surface_area_within_donut,
            'Orange surface area in remaining cytoplasm': orange_surface_area_without_donut,
            'Orange-Red Overlap surface area in all cell': orange_red_overlap_surface_area,
            'Orange-Red Overlap surface area around inclusions': orange_red_overlap_surface_area_within_donut,
            'Orange-Red Overlap surface area in remaining cytoplasm': orange_red_overlap_surface_area_without_donut
        })
        cell_count += 1

    df_cell_summary = pd.DataFrame(data)
    return df_cell_summary

def no_dox_analysis(red:np.ndarray, orange:np.ndarray, green:np.ndarray, path:str) -> pd.DataFrame:
    data = []
    df_cell_summary = pd.DataFrame()

    print(f"Starting analysis on {path}")


    contrast_adjusted_red_normalized = (red - red.min()) / (red.max() - red.min())
    threshold_value_red= np.mean(contrast_adjusted_red_normalized) + (np.std(contrast_adjusted_red_normalized) * 3) #1.75 works well
    red_thresholded = contrast_adjusted_red_normalized > threshold_value_red
    red_thresholded = remove_small_objects(red_thresholded, min_size=10)
    #display_two_images(red, red_thresholded, "Red Channel", "Red Thresholded", path)


    #contrast_adjusted_blue_normalized = (blue - blue.min()) / (blue.max() - blue.min())
    #threshold_value_blue= np.mean(contrast_adjusted_blue_normalized) + (np.std(contrast_adjusted_blue_normalized) * 1.5) #1.75 works well
    #blue_thresholded = contrast_adjusted_blue_normalized > threshold_value_blue
    #blue_thresholded = remove_small_objects(blue_thresholded, min_size=10)
    if ('mitotracker_farred_FCCP' in path): 
        contrast_adjusted_orange_normalized = (orange - orange.min()) / (orange.max() - orange.min())
        threshold_value_orange= np.mean(contrast_adjusted_orange_normalized) + (np.std(contrast_adjusted_orange_normalized) * 4) #1.75 works well
        orange_thresholded = contrast_adjusted_orange_normalized > threshold_value_orange
        orange_thresholded = remove_small_objects(orange_thresholded, min_size=10)
    else:
        threshold_value_orange  = threshold_otsu(orange)
        orange_thresholded = orange > threshold_value_orange

    data.append({
        'Image Path': path,
        'Red Surface Area in all cell': calculate_surface_area(label(red_thresholded)),
        'Orange Surface Area in all cell': calculate_surface_area(label(orange_thresholded)),
    })

    df_cell_summary = pd.DataFrame(data)
    return df_cell_summary

In [10]:
def main(image_folder):
    images_to_analyze = extract_image_paths(image_folder)
    output_dir = os.getcwd()
    df_cell_summary_list_dox = []
    df_cell_summary_list_nodox = []

    for path in images_to_analyze:
        image = read_image(path)
        image_squeezed = np.squeeze(image) 
    
        if 'mitotracker' in path.lower():
            red, orange, green = extract_channels(image_squeezed)
        else:
            orange, green, red = extract_channels(image_squeezed)

        if ('nodox' in path.lower()):
            df_cell_summary = no_dox_analysis(red, orange, green, path)
            df_cell_summary_list_nodox.append(df_cell_summary)
        else:
            df_cell_summary = dox_analysis(red, orange, green, path)
            df_cell_summary_list_dox.append(df_cell_summary)

    combined_cell_summary_df = pd.concat(df_cell_summary_list_dox, ignore_index=True)
    output_summary_path = os.path.join(output_dir, '61825_SUMMARY_DOX.xlsx')
    combined_cell_summary_df.to_excel(output_summary_path, index=False)

    combined_cell_summary_df_nodox = pd.concat(df_cell_summary_list_nodox, ignore_index=True)
    output_summary_path_nodox = os.path.join(output_dir, '61825_SUMMARY_NODOX.xlsx')
    combined_cell_summary_df_nodox.to_excel(output_summary_path_nodox, index=False)

if __name__ == "__main__":
    image_folder = 'images'
    main(image_folder)

Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_01.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_02.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_03.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_04.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_05.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_06.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_07.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_08.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_FCCP_01.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_FCCP_02.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_FCCP_03.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_FCCP_04.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotracker_farred_FCCP_05.czi
Starting analysis on images\3K_dox_LAMP1_RFP_mitotra